

<figure>
   <IMG SRC="https://avatars2.githubusercontent.com/u/31697400?s=400&u=a5a6fc31ec93c07853dd53835936fd90c44f7483&v=4" WIDTH=125 ALIGN="right">
</figure>
    
# Local grid refinement   

This notebook shows an example of an unstructured model create with the `nlmod` package.

### Table of contents
1. [Create model](#create)
2. [Run model](#run)
3. [Visualise](#Visualise)

In [1]:
import matplotlib.pyplot as plt
import flopy
import os
import geopandas as gpd

import nlmod

print(f'nlmod version: {nlmod.__version__}')

nlmod version: 0.0.1b


### 1. Create model<a name="create"></a>

Modflow 6 makes it possible to use locally refined grids. In NLmod you can use a shapefile and a number of levels to specify where and how much you want to use local grid refinement. Below we use a shapefile of the Planetenweg in IJmuiden and set the refinement levels at 2. This well create a grid with cells of 100x100m except at the Planetenweg where the cells will be refined to 25x25m. See also figures below.

Note: the `gen_model_unstructured` function also returns the `gridprops` dictionary. This contains data that is necesary to create some packages. It can be time consuming to generate this data.

In [2]:
# model settings unstructured
model_ws = 'model2'
model_name = 'IJm_planeten'
refine_shp_fname = os.path.join(nlmod.nlmod_datadir, 'shapes', 'planetenweg_ijmuiden')
levels=2
use_cache=False
extent=[95000., 105000., 494000., 500000.]
delr=100.
delc=100.
steady_state=False
steady_start=True
transient_timesteps=5
perlen=1.
verbose=True
start_time='2015-1-1'
gridtype = 'unstructured'
use_regis=True
regis_botm_layer=b'MSz1'
use_geotop=True
add_northsea=True
starting_head=1.0

In [ ]:
# Model directories
figdir, cachedir, gridgen_ws = nlmod.util.get_model_dirs(model_ws, gridtype=gridtype)


# create model time dataset
model_ds = nlmod.mdims.get_model_ds_time(model_name, model_ws, start_time,
                                         steady_state, steady_start=steady_start,
                                         transient_timesteps=transient_timesteps,
                                         perlen=perlen)

# create modflow packages
sim, gwf = nlmod.mfpackages.sim_tdis_gwf_ims_from_model_ds(model_ds,
                                                           verbose=verbose)

extent, nrow, ncol = nlmod.read.regis.fit_extent_to_regis(extent,
                                                          delr,
                                                          delc, verbose=verbose)

# layer model
layer_model = nlmod.read.regis.get_layer_models(extent, delr, delc,
                                                use_regis=use_regis,
                                                regis_botm_layer=regis_botm_layer,
                                                use_geotop=use_geotop,
                                                cachedir=cachedir,
                                                fname_netcdf='combined_layer_ds.nc',
                                                use_cache=use_cache,
                                                verbose=verbose)


creating modflow SIM, TDIS, GWF and IMS
redefining current extent: [95000.0, 105000.0, 494000.0, 500000.0], fit to regis raster
new extent is [95000.0, 105000.0, 494000.0, 500000.0] model has 60 rows and 100 columns
creating and caching dataset combined_layer_ds.nc



### Local grid refinement

the code below applies a local grid refinement to the layer model. The local grid refinement is based on the shapefile 'planetenweg_ijmuiden.shp', which contains a line shape of the Planetenweg, and the levels, which is 2. This means that the model cells at the Planetenweg will get a size of 25 x 25m. 

In [ ]:
# use gridgen to create unstructured grid
gridprops = nlmod.mdims.create_unstructured_grid(gridgen_ws, model_name, gwf,
                                                 [(refine_shp_fname, 'line', levels)], 
                                                 extent,
                                                 layer_model.dims['layer'],
                                                 nrow, ncol,
                                                 delr, delc,
                                                 cachedir=cachedir, use_cache=use_cache,
                                                 verbose=verbose)

# add layer model to unstructured grid
layer_model_unstr = nlmod.mdims.get_ml_layer_dataset_unstruc(raw_ds=layer_model,
                                                            extent=extent,
                                                            gridprops=gridprops,
                                                            cachedir=cachedir,
                                                            fname_netcdf='layer_model_unstr.nc',
                                                            use_cache=use_cache,
                                                            verbose=verbose)

# combine model time dataset with layer model dataset
model_ds = nlmod.mdims.update_model_ds_from_ml_layer_ds(model_ds,
                                                          layer_model_unstr,
                                                          gridtype,
                                                          keep_vars=['x', 'y'],
                                                          gridprops=gridprops,
                                                          add_northsea=add_northsea,
                                                          verbose=verbose)

# Create discretization
disv = nlmod.mfpackages.disv_from_model_ds(model_ds, gwf, gridprops)

In [ ]:
# create node property flow
nlmod.mfpackages.npf_from_model_ds(model_ds, gwf)

# Create the initial conditions package
nlmod.mfpackages.ic_from_model_ds(model_ds, gwf, starting_head=starting_head)

# Create the output control package
oc = nlmod.mfpackages.oc_from_model_ds(model_ds, gwf)

In [ ]:
# voeg grote oppervlaktewaterlichamen toe
da_name = 'surface_water'
model_ds = nlmod.mfpackages.surface_water.get_general_head_boundary(model_ds,
                                                                    gwf.modelgrid,
                                                                    da_name,
                                                                    cachedir=cachedir,
                                                                    use_cache=use_cache,
                                                                    verbose=verbose)
ghb = nlmod.mfpackages.ghb_from_model_ds(model_ds, gwf, da_name)

# surface level drain
model_ds = nlmod.read.ahn.get_ahn_dataset(model_ds, gridprops=gridprops,
                                          use_cache=use_cache,
                                           cachedir=cachedir, verbose=verbose)

drn = nlmod.mfpackages.surface_drain_from_model_ds(model_ds, gwf)



# add constant head cells at model boundaries
chd = nlmod.mfpackages.chd_at_model_edge_from_model_ds(model_ds, gwf, head='starting_head')

In [ ]:
# add knmi recharge to the model datasets
model_ds = nlmod.read.knmi.get_recharge(model_ds,
                                  verbose=verbose,
                                  cachedir=cachedir,
                                  use_cache=use_cache)
# create recharge package
rch = nlmod.mfpackages.rch_from_model_ds(model_ds, gwf)

In [ ]:
# save model_ds
model_ds.to_netcdf(os.path.join(cachedir, 'full_model_ds.nc'))

### 2. Run<a name="run"></a>
The model that we've created has not been runned yet. When you want to save and run the model you can use the code below.

Note: You always have to write the modflow data to the model workspace before you can run the model. 

In [ ]:
gwf.simulation.write_simulation()
gwf.simulation.run_simulation()

### 3. Visualise<a name="visualise"></a>

Using the `model_ds` and `gwf` variables it is quite easy to visualise model data. Below the modelgrid together with the surface water is shown.

In [ ]:
plan_weg_gdf= gpd.read_file(refine_shp_fname+'.shp')

# plot modelgrid
ax = nlmod.plots.plot_modelgrid(model_ds, gwf)
plan_weg_gdf.plot(ax=ax, color='r', label='Planetenweg')
ax.legend()
ax.get_figure().savefig(os.path.join(figdir, 'mgrid_swater.png'), bbox_inches='tight')

# plot zoomed modelgrid
ax = nlmod.plots.plot_modelgrid(model_ds, gwf)
ax.set_title('Planetenweg')
plan_weg_gdf.plot(ax=ax, color='r', label='Planetenweg')
ax.set_xlim(100000, 103000)
ax.set_ylim(495000,497500)
ax.legend()
ax.get_figure().savefig(os.path.join(figdir, 'Planetenweg.png'), bbox_inches='tight')

The model dataset of an unstructured model differs from a structured model dataset. The data is stored relative to the cell-id instead of the row and column number. Therefore the model dataset has the dimension cid instead of the dimensions x and y. 

In [ ]:
model_ds

To plot the same rasters as for the previous model we can use the `nlmod.plots.plot_array()` function.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14,11))
nlmod.plots.plot_array(gwf, model_ds['ahn'], ax=axes[0][0])
nlmod.plots.plot_array(gwf, model_ds['bot'][0],ax=axes[0][1])
nlmod.plots.plot_array(gwf, model_ds['idomain'][0],ax=axes[1][0])
nlmod.plots.plot_array(gwf, model_ds['chd'][0],ax=axes[1][1])
for axes1 in axes:
    for ax in axes1:
        ax.axis('scaled')

fig.savefig(os.path.join(figdir, 'ahn_bot_idom_chd.png'), bbox_inches='tight')        
        
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14,11))
nlmod.plots.plot_array(gwf, model_ds['bathymetry'],ax=axes[0][0])
nlmod.plots.plot_array(gwf, model_ds['northsea'],ax=axes[0][1])
nlmod.plots.plot_array(gwf, model_ds['kh'][1],ax=axes[1][0])
nlmod.plots.plot_array(gwf, model_ds['recharge'][:,0],ax=axes[1][1])

for axes1 in axes:
    for ax in axes1:
        ax.axis('scaled')
fig.savefig(os.path.join(figdir, 'bath_nsea_kh_rch.png'), bbox_inches='tight')        